In [ ]:
#Imports
import matplotlib.pyplot as plotter_lib

import numpy as np

import PIL as image_lib

import tensorflow as tflow

from tensorflow.keras.layers import Flatten

from keras.layers.core import Dense

from tensorflow.keras.models import Sequential

from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

from keras.layers import Dropout

import random


In [ ]:
#Dataset to learn

import pathlib

demo_dataset = pathlib.Path(r"C:\Users\micha\OneDrive\Desktop\Classification - Copy")

directory = str(demo_dataset)

data_directory = pathlib.Path(directory)

In [ ]:
#pre-proccesing and splitting the dataset

img_height,img_width=180,180

batch_size=32

train_ds = tflow.keras.preprocessing.image_dataset_from_directory(

  data_directory,

  validation_split=0.2,

  subset="training",

  seed=2,

label_mode='categorical',

  image_size=(img_height, img_width),

  batch_size=batch_size)

In [ ]:
#pre-proccesing and splitting the dataset

validation_ds = tflow.keras.preprocessing.image_dataset_from_directory(

  data_directory,

  validation_split=0.2,

  subset="validation",

  seed=123,

label_mode='categorical',

  image_size=(img_height, img_width),

  batch_size=batch_size)

In [ ]:
#Choosing the model- Sequential, Adding pre-trained model-ResNet50
#Choosing parameters of the model (dropout)

demo_resnet_model = Sequential()

pretrained_model_for_demo = tflow.keras.applications.ResNet50(include_top=False,
                                                              input_shape=(180,180,3),
                                                              pooling='avg',
                                                              classes=34, 
                                                              weights='imagenet')

for each_layer in pretrained_model_for_demo.layers:
    each_layer.trainable = False

demo_resnet_model.add(pretrained_model_for_demo)
demo_resnet_model.add(Flatten())
demo_resnet_model.add(Dense(512, activation='relu'))
demo_resnet_model.add(Dropout(0.5))  # Dropout layer with rate of 0.5
demo_resnet_model.add(Dense(34, activation='softmax'))


In [ ]:
# Higher the amount the more the model learns around 200 achieved some good results.
epochs=200

In [ ]:
#Train the model 
#choose optimizer
#choose metrics
demo_resnet_model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

history = demo_resnet_model.fit(train_ds, validation_data=validation_ds, epochs=epochs)

In [ ]:
#Visualizing the training and validation accuracy
#checking the loss to determine the amount of Epochs needed

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#printing all class names
class_names = train_ds.class_names

print("Classification Labels:")
for i, label in enumerate(class_names):
    print(f"{i}: {label}")


In [ ]:
# Load a random image from the validation dataset
for image_batch, label_batch in validation_ds:
    random_index = random.randint(0, len(image_batch)-1)
    image = image_batch[random_index]
    true_labels = label_batch[random_index]
    break

# Display the chosen image
plt.imshow(image.numpy().astype("uint8"))
plt.axis("off")
plt.show()

# Preprocess the image
img_array = np.array(image)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

# Use the model to predict the classication of the imageÇ
predictions = demo_resnet_model.predict(img_array)
predicted_labels = np.where(predictions > 0.1)[1]

# Print the true labels
true_label_names = [class_names[i] for i in np.where(true_labels == 1)[0]]
print("True labels:", true_label_names)

# Print the predicted labels
predicted_label_names = [class_names[i] for i in predicted_labels]
print("Predicted labels:", predicted_label_names)


In [ ]:
#In case you have a test dataset

test_path = r"C:\Users\micha\OneDrive\Desktop\Test"

# Load a random image from the test dataset
images = os.listdir(test_path)
random_index = random.randint(0, len(images)-1)
image_path = os.path.join(test_path, images[random_index])
image = cv2.imread(image_path)

# Display the chosen image
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

# Preprocess the image
img_array = cv2.resize(image, (224, 224))
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

# Use the model to predict the class of the image
predictions = demo_resnet_model.predict(img_array)
predicted_labels = np.where(predictions > 0.2)[1]

# Print the predicted labels
predicted_label_names = [class_names[i] for i in predicted_labels]
print("Predicted labels:", predicted_label_names)

